## Setup & read in data

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt   

In [ ]:
#this is the data including imputed values for test results etc 
all_data = pd.read_csv('imputed_all_data.csv')
parse_dates=['admission_date_structured']
labels = pd.read_csv('new_dbscan_9clus.csv')
combined_clean = pd.read_csv('combined_clean.csv')


In [ ]:
orig_data_and_labels_comorb = pd.merge(all_data, labels,
                           how='right', on='id')

In [ ]:
#remove comorbidities 
orig_data_and_labels = orig_data_and_labels_comorb.drop(columns=['Unnamed: 0','morbidity_Diabetes','morbidity_COPD','morbidity_Hypertension','morbidity_Heartdisease','morbidity_Renaldisease','morbidity_Tumor','morbidity_Metabolicdisorders','morbidity_Respiratorydiseases'])

In [ ]:
orig_data_and_labels

,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,SystolicBP_d1_min,...,AlbumintoGlobulinRatio,Male,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster
0,100251,155.00000,60.000000,0,62.0,4.660000,4.660000,4.925083,5.956664,124.000000,...,1.382490,False,1.0,36.6,20.0,12.100000,134.500000,68.500000,23.384615,0
1,100358,164.75961,65.000000,0,62.0,5.793085,6.349398,5.576803,5.064091,115.000000,...,1.006937,True,1.0,37.5,24.0,11.900000,122.500000,80.000000,15.538462,0
2,101536,158.00000,50.000000,0,48.0,5.248720,6.629004,4.899923,5.805336,98.000000,...,1.437422,False,1.0,36.7,22.0,11.300000,108.000000,62.500000,46.326531,0
3,101289,165.00000,60.000000,0,50.0,4.458859,5.799471,3.930688,4.154902,118.000000,...,1.329695,False,1.0,36.5,20.0,13.600000,122.000000,73.000000,22.432432,0
4,100851,170.00000,70.000000,0,74.0,5.270000,5.270000,4.348679,5.624614,116.000000,...,1.283246,True,1.0,36.8,24.0,12.700000,121.500000,81.000000,27.342448,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,102301,175.00000,72.000000,0,67.0,4.110000,4.110000,5.335332,5.082172,114.000000,...,1.240000,True,1.0,36.5,28.0,14.600000,120.000000,74.500000,20.384615,8
2756,101680,164.75961,65.029389,1,77.0,5.680000,5.680000,6.280000,6.280000,98.000000,...,1.240000,True,0.0,37.2,18.0,16.400000,118.500000,63.500000,5.454545,8
2757,102631,168.00000,65.000000,0,75.0,8.380000,8.380000,6.140000,6.140000,130.000000,...,1.700000,True,1.0,36.3,23.0,12.500000,140.000000,70.000000,8.358209,8
2758,100166,164.75961,65.029389,0,69.0,4.133798,7.149667,11.590000,11.590000,121.000000,...,1.120000,True,1.0,38.2,23.0,11.700000,134.000000,83.000000,33.750000,8


### Setup: multiclass

In [ ]:
orig_data_and_labels.cluster.value_counts()

1    1114
0     914
3     255
4     232
2     112
6      44
5      43
7      28
8      18
Name: cluster, dtype: int64

Remove the two largest clusters; these are the non-severe patients 

In [ ]:
num_to_remove = 2 
# Remove the largest n clusters to focus on interesting ones 

value_counts = orig_data_and_labels.cluster.value_counts()
value_counts_list = value_counts.index.to_list()
interesting_clusters = value_counts_list[num_to_remove:len(value_counts)]

data_and_labels = orig_data_and_labels.loc[orig_data_and_labels['cluster'].isin(interesting_clusters)]

In [ ]:
data_and_labels.cluster.value_counts()

3    255
4    232
2    112
6     44
5     43
7     28
8     18
Name: cluster, dtype: int64

In [ ]:
data_and_labels2 = data_and_labels.copy()
data_and_labels2['cluster'] = data_and_labels2['cluster'].apply(lambda x: + x)

In [ ]:
from sklearn import preprocessing, model_selection

def preprocess(all_data):
#     Split the dataset into features and labels (clusters) - so we can normalise the features but not the labels
    all_X = all_data.iloc[:,2:len(all_data.columns)-1]
    all_y = all_data['cluster']
    
    # normalise the data so we have unit variance and mean 0 using built-in preprocessing method in sklearn
    scaler = preprocessing.StandardScaler()
    all_X= pd.DataFrame(scaler.fit_transform(all_X),columns=all_X.columns)
    
#     reset the indexes otherwise it was breaking
    all_X= all_X.reset_index()
    all_y = all_y.reset_index()
    all_X = all_X.drop(columns=['index'])
    all_y = all_y.drop(columns=['index'])
    
    #prepare data for training a model by splitting into training and testing data 
    return (all_X, all_y)

In [ ]:
all_X, all_y = preprocess(data_and_labels2)

In [ ]:
data_and_labels2

,id,Height,Weight,smoking_structured,age,blood_sugar_d1_min,blood_sugar_d1_max,blood_sugar_d3_min,blood_sugar_d3_max,SystolicBP_d1_min,...,AlbumintoGlobulinRatio,Male,Consciousness,Temperature,Respiratoryrate,Redcelldistributionwidth,SystolicBP,DiastolicBP,Lymphocyte(%),cluster
2028,102710,176.00000,70.000000,0,78.0,4.970000,4.970000,4.795946,4.265159,136.000000,...,1.431469,True,1.0,36.3,20.0,12.900000,144.000000,81.500000,24.917824,2
2029,101282,164.75961,65.029389,0,70.0,5.195489,4.721443,5.327653,4.032294,134.000000,...,1.237522,True,1.0,38.0,24.0,12.200000,139.500000,79.500000,14.920635,2
2030,100199,164.75961,60.000000,0,60.0,4.777269,4.160881,5.300442,6.900181,114.000000,...,1.434636,False,0.0,37.0,24.0,14.000000,125.000000,75.500000,24.528302,2
2031,101283,160.00000,73.000000,0,64.0,4.706814,6.641539,8.390000,8.390000,116.000000,...,1.439448,False,0.0,36.7,24.0,12.500000,124.000000,74.000000,10.512821,2
2032,100362,160.00000,80.000000,0,56.0,4.167717,5.796114,4.198280,6.417327,120.000000,...,1.134994,False,1.0,37.5,22.0,12.771874,121.000000,77.000000,29.363816,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,102301,175.00000,72.000000,0,67.0,4.110000,4.110000,5.335332,5.082172,114.000000,...,1.240000,True,1.0,36.5,28.0,14.600000,120.000000,74.500000,20.384615,8
2756,101680,164.75961,65.029389,1,77.0,5.680000,5.680000,6.280000,6.280000,98.000000,...,1.240000,True,0.0,37.2,18.0,16.400000,118.500000,63.500000,5.454545,8
2757,102631,168.00000,65.000000,0,75.0,8.380000,8.380000,6.140000,6.140000,130.000000,...,1.700000,True,1.0,36.3,23.0,12.500000,140.000000,70.000000,8.358209,8
2758,100166,164.75961,65.029389,0,69.0,4.133798,7.149667,11.590000,11.590000,121.000000,...,1.120000,True,1.0,38.2,23.0,11.700000,134.000000,83.000000,33.750000,8


## Classification setup

In [ ]:
#Set the class names
class_names = data_and_labels2['cluster'].unique()

### setting up performance metrics calculations

In [ ]:
from sklearn import metrics
#method to compute performance metrics 
def performance_metrics(y_true, y_pred):
    #create a confusion matrix from results 
    cnf_matrix  = metrics.confusion_matrix(y_true, y_pred, labels=class_names, sample_weight=None)
#     # average over all classes to find overall fp,fn,tp,tn
#     FP = np.sum(cnf_matrix, axis=0) - np.diag(cnf_matrix)
#     FN = np.sum(cnf_matrix, axis=1) - np.diag(cnf_matrix)
#     TP = np.diag(cnf_matrix)
#     TN = np.sum(cnf_matrix) - (FP + FN + TP)
        
    #find out if each predicted label is right 
    correct_labels = 0
    for i, true_label in enumerate(y_true):
        if (true_label == y_pred[i]):
            correct_labels += 1

    #calculate overall accuracy
#     acc = np.round(correct_labels/y_pred.shape[0],2)
        
    # tp rate is the same as recall 
#     recall = np.sum(TP)/(np.sum(TP)+np.sum(FN))
#     FP_rate = np.sum(FP)/(np.sum(FP)+np.sum(TN))  
#     precision = np.sum(TP)/(np.sum(TP)+np.sum(FP)) 
#     f_measure = (2*precision*recall)/(precision + recall) 

    metrics_dict = metrics.classification_report(y_true,y_pred,output_dict=True)
    avg_metrics = metrics_dict['macro avg']
    
    recall = avg_metrics['recall']
    precision = avg_metrics['precision']
    f_measure = avg_metrics['f1-score']
    acc = metrics_dict['accuracy']
    
    print(metrics.classification_report(y_true,y_pred))
    
    return (acc, recall, precision, f_measure,
            np.round(cnf_matrix/cnf_matrix.sum(axis=1), 2))


In [ ]:
import seaborn as sn

#method to plot the confusion matrix 
def plot_confusion_matrix(confusion_matrix, class_names):
    conf_matrix = pd.DataFrame(confusion_matrix, index = [i for i in class_names], columns = [i for i in class_names])
    plt.figure()
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(conf_matrix, annot=True, annot_kws={"size": 16}, cmap="Blues")
    plt.show()

## k-fold cross validation

Here we use k-fold cross validation to evaluate the effectiveness of our models, where each 'fold' results in a slightly different train/test split, allowing us to see how the model beaves with different data. We can compute various metrics about the model performance, and from these results, we can choose the most effective classification model.

In [ ]:
from sklearn import model_selection 
from sklearn import linear_model
from sklearn import ensemble

def cross_val(all_X, all_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing):
    print('cross-validating...')
    k_val = 3
    #use 3 splits 
    k_fold = model_selection.KFold(n_splits=k_val)
    strat_k_fold = model_selection.StratifiedKFold(n_splits=k_val, shuffle=True)

    #create empty arrays to store the results from each fold
    #we can then average these to get the overall classifications and performance 
    accuracies = np.empty(k_val)
    tp_rates = np.empty(k_val)
    precisions = np.empty(k_val)
    f_measures = np.empty(k_val)

    count = 0

    #split training data into training and validation sets
    for train_indices, validation_indices in strat_k_fold.split(all_X, all_y):
    #     print(X_train.index)
        X_training = all_X.iloc[train_indices]
        y_training = all_y.iloc[train_indices]
        X_validate = all_X.iloc[validation_indices]
        y_validate = all_y.iloc[validation_indices]
        
        # upsample the training data in each fold, if upsample = true
        #with stratified k fold, better not to upsample, as it preserves percentage of samples for each class 
#         print('value counts: ', y_training.value_counts())

        if(downsampling):
#             print('downsampling...')
            X_training, y_training = downsample(X_training, y_training, class_names, down_prop)
            if (printing):
                print('value counts: ', y_training.value_counts())

        if (upsampling):
#             print('upsampling...')
            X_training, y_training = upsample(X_training, y_training, class_names, up_prop)
            if (printing):
                print('value counts:', y_training.value_counts())
            
        # fit classifier
        clf.fit(X_training, y_training.values.ravel())
        
        #predict 
        y_predicted = clf.predict(X_validate)
        y_true_val = y_validate.cluster
        
        #if we are using the decision tree classifier we may want to export the tree to look at it 
        from sklearn import tree

        export_tree = False
        if(export_tree):
            for tree_in_forest in clf.estimators_:
                if(export_tree):
                    tree.export_graphviz(tree_in_forest, out_file="rotated_tree_eg.dot",filled = True, 
                                         feature_names = X_training.columns, class_names=class_names,
                                         rotate = True)
        if (printing):
            print(np.unique(y_predicted, return_counts=True))

        #see how validation set performs 
        acc, recall, precision, f_measure, confusion_matrix = performance_metrics(y_true_val.values, y_predicted)

        if (printing):
            #print performance results for each fold
            print("accuracy: ", acc)
            print("true positive rate / recall: ", recall)
            print("precision: ", precision)
            print("f measure: ", f_measure)
            plot_confusion_matrix(confusion_matrix, class_names)
            print("----------")

        # store info for each fold 
        accuracies[count] = acc
        tp_rates[count] = recall
        precisions[count] = precision
        f_measures[count] = f_measure
        count+=1
        
        
    #average scores
    cv_acc = np.mean(accuracies)
    cv_tp = np.mean(tp_rates)
    cv_precision = np.mean(precisions)
    cv_f_measure = np.mean(f_measures)


    print("overall average performance:")
    print("accuracy: ", cv_acc)
    print("true positive rate/recall: ", cv_tp)
    print("precision: ", cv_precision)
    print("f measure: ", cv_f_measure)
    
    return X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure


Method to perform grid search to find the best parameters for the random forest classifier, uses cross-val


Had to implement this manually due to library constraints and difficulties with upsampling properly inside gridsearch

In [ ]:
from sklearn import model_selection, pipeline, ensemble

def grid_search_rand_forest(data_X, data_y,class_names, downsampling, upsampling, down_prop, up_prop):

    best_f = 0.0
    
    # gridsearch for parameter optimisation
    param_grid = { 
        'n_estimators': [5, 10, 15, 20, 30, 40, 60, 80],
        'max_depth': [5, 10, 15, None],
        'max_features': ['auto', 'sqrt', 'log2'],
        'min_samples_split': [2, 5, 10],
        'criterion' :['gini', 'entropy']}
    
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for max_features in param_grid['max_features']:
                for min_samples_split in param_grid['min_samples_split']:
                    for criterion in param_grid['criterion']:
                        clf = ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                                               min_samples_split=min_samples_split, criterion=criterion)

                        X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, f_measure = cross_val(data_X, data_y, class_names, downsampling, upsampling, down_prop, up_prop, clf, printing=False)

                        if(f_measure > best_f):
                            best_f = f_measure
                            best_clf = clf
    
    # Create grid search object
#     clf = model_selection.GridSearchCV(estimator=ensemble.RandomForestClassifier(), param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

#     best_clf = clf.fit(data_X, data_y.values.ravel())

#     gridsearch_results = pd.DataFrame(best_clf.cv_results_)

    #sort according to overall rank score to find best params
#     best_params_found = gridsearch_results.sort_values('rank_test_score').iloc[0]
    print(best_clf)
    
    return best_clf

## Rebalancing classes

### Downsampling

To address class imbalance by removing some examples in most numerous classes

In [ ]:
def downsample(train_X, train_y, class_names, down_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y
    
    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)

    most_numerous =  train_data_and_labels.cluster.value_counts().index[0]
    big_cluster = train_data_and_labels[train_data_and_labels['cluster']==most_numerous]
    new_train = train_data_and_labels[train_data_and_labels.cluster!=most_numerous]
    #remove more individuals in largest class if limited classes, as class imbalance is bigger here
    reduced_cluster = big_cluster.sample(replace=False, n=int(len(big_cluster)*down_prop), random_state=1)
    train_down = new_train.append(reduced_cluster)
    y_train_down = train_down['cluster']
    X_train_down = train_down.iloc[:,0:len(train_down.columns)-1]        
    
    return X_train_down, y_train_down

### Upsampling 

This improves the prediction accuracy for the classes containing fewer individuals.

In [ ]:
#upsample the least numerous classes
def upsample(train_X, train_y, class_names, up_prop):
    
    train_data_and_labels = train_X.copy()
    train_data_and_labels['cluster'] = train_y

    #find the largest class and the number of samples in training
    value_counts = train_data_and_labels.cluster.value_counts()
    max_val = max(value_counts)
        
    #upsampling all the classes in training data 
    for cluster in class_names:
        cluster_vals = train_data_and_labels[train_data_and_labels.cluster==int(cluster)]
        num_in_cluster = len(cluster_vals.index)
        num_extra_samples = int(up_prop*(max_val - num_in_cluster))
        new_samples = cluster_vals.sample(replace=True, n=num_extra_samples, random_state=1)
        train_data_and_labels = train_data_and_labels.append(new_samples)
            
#     print(train_data_and_labels.cluster.value_counts())
    
    X_train_up = train_data_and_labels.iloc[:,0:len(train_data_and_labels.columns)-1]
    y_train_up = train_data_and_labels['cluster']
    
    return X_train_up, y_train_up

Set parameters

In [ ]:
#need to do gridsearch 
# clf = grid_search_rand_forest(all_X, all_y, class_names, downsampling=True, upsampling=True, down_prop=1, up_prop=0.8)
clf = ensemble.RandomForestClassifier(criterion='entropy', max_depth=15, max_features='log2',
                       n_estimators=60)

We now have a new dataframe containing a more equal distribution of each class to work with. 

Now we perform the k-fold cross validation on data. Since our dataset is small, the predictions can vary a lot based on which data is used for training/testing, which is why k-fold cross-val is useful to get overall prediction metrics. 
If upsampling is true then the least numerous classes will be upsampled.

In [ ]:
class_names = all_y['cluster'].unique()
X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure = cross_val(all_X, all_y, class_names, 
                                                           downsampling = True, upsampling=True,
                                                           down_prop = 1, up_prop = 0.8, clf=clfSevere, printing=False)


cross-validating...
              precision    recall  f1-score   support

           2       0.43      0.42      0.43        38
           3       0.48      0.53      0.50        85
           4       0.41      0.43      0.42        77
           5       0.21      0.29      0.24        14
           6       0.50      0.07      0.12        15
           7       0.29      0.22      0.25         9
           8       0.50      0.33      0.40         6

    accuracy                           0.42       244
   macro avg       0.40      0.33      0.34       244
weighted avg       0.43      0.42      0.41       244

              precision    recall  f1-score   support

           2       0.38      0.35      0.37        37
           3       0.45      0.49      0.47        85
           4       0.40      0.45      0.42        78
           5       0.23      0.21      0.22        14
           6       0.00      0.00      0.00        15
           7       0.40      0.44      0.42         9
    

#### Multiclass:

Now let's try to look at only the most severe clusters (5,7,8) , and see if we can distinguish between these


create new dataframe for severe clusters

In [ ]:
c5 = data_and_labels2[data_and_labels2.cluster == 5]
c7 = data_and_labels2[data_and_labels2.cluster == 7]
c8 = data_and_labels2[data_and_labels2.cluster == 8]

cSevere = pd.concat([c5,c7])
cSevere = pd.concat([cSevere,c8])

In [ ]:
subchapters_all_XSevere, subchapters_all_ySevere = preprocess(cSevere)

hyperparameters 

In [ ]:
class_names=subchapters_all_ySevere.cluster.unique()
# clfSevere = grid_search_rand_forest(subchapters_all_XSevere, subchapters_all_ySevere,class_names, downsampling=False, upsampling=True, down_prop=1, up_prop=0.8)
clfSevere = ensemble.RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=20)

### Prediction

In [ ]:

class_names = subchapters_all_ySevere['cluster'].unique()
X_training, y_training, X_validate, y_validate, y_predicted, y_true_val, cv_f_measure = cross_val(subchapters_all_XSevere, subchapters_all_ySevere, class_names, 
                                                           downsampling = True, upsampling=True,
                                                           down_prop = 1, up_prop = 0.8, clf=clfSevere, printing=False)

cross-validating...
              precision    recall  f1-score   support

           5       0.67      0.80      0.73        15
           7       0.55      0.67      0.60         9
           8       1.00      0.17      0.29         6

    accuracy                           0.63        30
   macro avg       0.74      0.54      0.54        30
weighted avg       0.70      0.63      0.60        30

              precision    recall  f1-score   support

           5       0.76      0.93      0.84        14
           7       0.86      0.60      0.71        10
           8       0.67      0.67      0.67         6

    accuracy                           0.77        30
   macro avg       0.76      0.73      0.74        30
weighted avg       0.78      0.77      0.76        30

              precision    recall  f1-score   support

           5       0.58      0.79      0.67        14
           7       0.57      0.44      0.50         9
           8       0.33      0.17      0.22         6

